<a href="https://colab.research.google.com/github/akanison12/Sudo-code-2024/blob/main/240114_FineTuningBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install liblary


In [ ]:
! pip install -q kaggle
!kaggle datasets download nkhachao/vietnamese-squad
! unzip vietnamese-squad.zip
! pip install transformers
! pip install datasets




In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer,AutoModelForQuestionAnswering,Trainer,TrainingArguments
import torch
from torch.utils.data import Dataset,DataLoader
import json









#Read data


In [ ]:
train_file='/content/train-v2.0-translated.json'
val_file='/content/dev-v2.0-translated.json'

with open(train_file,'rb') as f:
  raw_train=json.load(f)

with open(val_file,'rb') as f:
  raw_val=json.load(f)







In [ ]:
raw_train[0]




['Beyoncé Giselle Knowles-Carter (/ b i gì ɒ n s eɪ / bee-YON-say) (sinh ngày 04 tháng 9 1981) là một ca sĩ, nhạc sĩ, nhà sản xuất thu âm và nữ diễn viên người Mỹ. Sinh ra và lớn lên ở Houston, Texas, cô đã biểu diễn trong các cuộc thi ca hát và nhảy múa khác nhau khi còn nhỏ, và nổi tiếng vào cuối những năm 1990 với tư cách là ca sĩ chính của nhóm nhạc nữ R & B Destiny\'s Child. Được quản lý bởi cha cô, Mathew Knowles, nhóm đã trở thành một trong những nhóm nhạc nữ bán chạy nhất thế giới mọi thời đại. Sự gián đoạn của họ đã chứng kiến việc phát hành album đầu tay của Beyoncé, Dangerously in Love (2003), giúp cô trở thành một nghệ sĩ solo trên toàn thế giới, giành được năm giải Grammy và có đĩa đơn quán quân Billboard Hot 100 "Crazy in Love" và "Baby Boy".',
 'Beyonce bắt đầu nổi tiếng từ khi nào?',
 'Vào cuối những năm 1990']

#Create dataset

In [ ]:
def create_Dataset(data):
  """
    Creates a structured dataset with content, question, and answer information.

    Args:
        data (list): A list of tuples, where each tuple contains:
                     (content, question, answer).

    Returns:
        list: A processed list of dictionaries with answer details and positions.
    """

    dataset=[]
    for datasub in data:
        content=datasub[0]
        question=datasub[1]
        answer=datasub[2]

        #Find answer_start
        answer_start=content.find(answer)
        #
        if answer_start==-1:
            continue
        #Find answer_start
        answer_end=answer_start+len(answer)
        #Append entry with necessary details to the dataset
        dataset.append({
            'content':content,
            'question':question,
            'answer':{
                'text':[answer],
                'answer_start':[answer_start],
                'answer_end':[answer_end]
            }

        })




    return dataset










In [ ]:
data_train=create_Dataset(raw_train)
data_val=create_Dataset(raw_val)



In [ ]:
data_train[0]

{'content': 'Beyoncé Giselle Knowles-Carter (/ b i gì ɒ n s eɪ / bee-YON-say) (sinh ngày 04 tháng 9 1981) là một ca sĩ, nhạc sĩ, nhà sản xuất thu âm và nữ diễn viên người Mỹ. Sinh ra và lớn lên ở Houston, Texas, cô đã biểu diễn trong các cuộc thi ca hát và nhảy múa khác nhau khi còn nhỏ, và nổi tiếng vào cuối những năm 1990 với tư cách là ca sĩ chính của nhóm nhạc nữ R & B Destiny\'s Child. Được quản lý bởi cha cô, Mathew Knowles, nhóm đã trở thành một trong những nhóm nhạc nữ bán chạy nhất thế giới mọi thời đại. Sự gián đoạn của họ đã chứng kiến việc phát hành album đầu tay của Beyoncé, Dangerously in Love (2003), giúp cô trở thành một nghệ sĩ solo trên toàn thế giới, giành được năm giải Grammy và có đĩa đơn quán quân Billboard Hot 100 "Crazy in Love" và "Baby Boy".',
 'question': "Beyonce rời Destiny's Child và trở thành ca sĩ solo từ khi nào?",
 'answer': {'text': ['2003'], 'answer_start': [602], 'answer_end': [606]}}

#Processing For Bert


In [ ]:
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
def prepare_data(data):
    """
    Processes input data into a structured format for model training, containing content,
    questions, and answer information with token positions.

    Args:
        data (dict): A dictionary with keys 'question', 'content', and 'answer',
                     where 'answer' contains 'answer_start' and 'answer_end'.

    Returns:
        dict: A dictionary with tokenized inputs, including start and end positions of the answer.
    """

    # Tokenize the question and content
    inputs = tokenizer(
        data['question'],
        data['content'],
        max_length=384,
        truncation="only_second",
        padding="max_length",
        return_offsets_mapping=True,
        return_tensors="pt"
    )

    # Extract start and end character positions of the answer
    start_char = data['answer']['answer_start'][0]
    end_char = data['answer']['answer_end'][0]

    # Determine the token positions for the answer
    offsets = inputs["offset_mapping"].squeeze(0)
    start_token = end_token = 0

    for idx, (start, end) in enumerate(offsets):
        if start <= start_char < end:
            start_token = idx
        if start < end_char <= end:
            end_token = idx
            break

    # Remove batch dimension from the inputs
    inputs['input_ids'] = inputs['input_ids'].squeeze(0)
    inputs['token_type_ids'] = inputs['token_type_ids'].squeeze(0)
    inputs['attention_mask'] = inputs['attention_mask'].squeeze(0)

    # Add start and end positions as tensors
    inputs['start_positions'] = torch.tensor([start_token], dtype=torch.long)
    inputs['end_positions'] = torch.tensor([end_token], dtype=torch.long)

    # Remove offset mapping from the final input
    inputs.pop("offset_mapping")

    return inputs






In [ ]:
from datasets import Dataset

# Convert training and validation data lists to Dataset objects
train_dataset = Dataset.from_list(data_train)
val_dataset = Dataset.from_list(data_val)


train_dataset = train_dataset.map(prepare_data, batched=False)
val_dataset = val_dataset.map(prepare_data, batched=False)



Map:   0%|          | 0/79198 [00:00<?, ? examples/s]

Map:   0%|          | 0/8359 [00:00<?, ? examples/s]

In [ ]:

train_dataset = train_dataset.remove_columns(["content", "question", "answer"])
val_dataset = val_dataset.remove_columns(["content", "question", "answer"])

In [ ]:
train_dataset.save_to_disk("/content/drive/MyDrive/ Sudo-ccode/W8")
val_dataset.save_to_disk("/content/drive/MyDrive/ Sudo-ccode/W8")

Saving the dataset (0/1 shards):   0%|          | 0/79198 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/8359 [00:00<?, ? examples/s]

#Create Model

In [ ]:
from datasets import load_from_disk


train_dataset = load_from_disk("/content/drive/MyDrive/ Sudo-ccode/W8")
val_dataset = load_from_disk("/content/drive/MyDrive/ Sudo-ccode/W8")

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_name)


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#The Trainer and  classes in Hugging Face's 🤗 Transformers library offer a comprehensive interface for training, fine-tuning, and evaluating models.



In [ ]:
#
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,1.090308
2,1.366100,0.759950
3,1.366100,0.625952


TrainOutput(global_step=786, training_loss=1.174312407127167, metrics={'train_runtime': 2500.1095, 'train_samples_per_second': 10.03, 'train_steps_per_second': 0.314, 'total_flos': 4914404076501504.0, 'train_loss': 1.174312407127167, 'epoch': 3.0})

#trainer.train() to train and trainer.evaluate() to evaluate.

#### Exact Match (EM) Formula
The **Exact Match (EM)** score is calculated using the following formula:

$$
\text{Exact Match (EM)} = \frac{\text{Number of exact matches}}{\text{Total number of samples}} \times 100
$$

- **Exact Match** is defined as a predicted answer matching the true answer exactly, including text but ignoring leading and trailing spaces.


In [ ]:
#EM(Exact_Match)
def compute_exact_match(pred_answer, true_answer):

    return int(pred_answer.strip() == true_answer.strip())


In [3]:


def evaluate_exact_match(model, eval_dataset, tokenizer):
  """
    Evaluates the exact match score for a model on a given evaluation dataset.

    Args:
        model: The model to be evaluated.
        eval_dataset (Dataset): The evaluation dataset containing input data and labels.
        tokenizer: The tokenizer used for decoding.

    Returns:
        dict: A dictionary containing the exact match score.
    """
    exact_matches = []


    for data in eval_dataset:
        input_ids = torch.tensor([data['input_ids']]).to(model.device)
        attention_mask = torch.tensor([data['attention_mask']]).to(model.device)


        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)


        start_logits = outputs.start_logits
        end_logits = outputs.end_logits


        start_token = torch.argmax(start_logits, dim=1).item()
        end_token = torch.argmax(end_logits, dim=1).item()


        predicted_answer = tokenizer.decode(
            input_ids[0][start_token:end_token + 1], skip_special_tokens=True
        )


        true_answer = tokenizer.decode(
            input_ids[0][data['start_positions'][0]:data['end_positions'][0] + 1], skip_special_tokens=True
        )


        exact_match = compute_exact_match(predicted_answer, true_answer)
        exact_matches.append(exact_match)


    exact_match_score = sum(exact_matches) / len(exact_matches) * 100

    return {"exact_match": exact_match_score}


#Metrices

In [ ]:
eval_results = evaluate_exact_match(model,val_dataset, tokenizer)
print("Evaluation Results - Exact Match:", eval_results["exact_match"])


Evaluation Results - Exact Match: 70.13996889580093


#Infenrence


In [ ]:



question = "Trường Đại học Khoa học Tự nhiên được thành lập vào năm nào?"
context = """
Trường Đại học Khoa học Tự nhiên, Đại học Quốc gia Thành phố Hồ Chí Minh (ĐHQG-HCM), là một trường đại học công lập
chuyên về khoa học cơ bản và ứng dụng. Được thành lập vào năm 1996, trường là một trong những trường đào tạo hàng đầu
trong lĩnh vực khoa học tại Việt Nam, với nhiều chương trình nghiên cứu và đào tạo tiên tiến. Trường có sứ mệnh
nâng cao kiến thức khoa học và ứng dụng, cung cấp nguồn nhân lực chất lượng cao cho đất nước.
"""


inputs = tokenizer(
    question,
    context,
    max_length=384,
    truncation="only_second",
    padding="max_length",
    return_tensors="pt"
)

#connect to GPU
inputs = {k: v.to(model.device) for k, v in inputs.items()}

#Infenrence
with torch.no_grad():
    outputs = model(**inputs)

#Find start,end
start_logits = outputs.start_logits
end_logits = outputs.end_logits


start_token = torch.argmax(start_logits, dim=1).item()
end_token = torch.argmax(end_logits, dim=1).item()


predicted_answer = tokenizer.decode(inputs["input_ids"][0][start_token:end_token + 1], skip_special_tokens=True)

print("Question:", question)
print("Predicted Answer:", predicted_answer)


Question: Trường Đại học Khoa học Tự nhiên được thành lập vào năm nào?
Predicted Answer: 1996


#Save Model

In [ ]:
# Save Token
tokenizer.save_pretrained("/content/drive/MyDrive/ Sudo-ccode/W8")

# Save model
model.save_pretrained("/content/drive/MyDrive/ Sudo-ccode/W8")